## Daten Laden

In [1]:
file_path = "LITTLE_2025-05-10__03_52_57-normalized-supplemented.csv"
print(file_path)

LITTLE_2025-05-10__03_52_57-normalized-supplemented.csv


In [2]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

,watt-median,L1-dcache-loads:u,L1-dcache-loads:u-square,L1-dcache-stores:u,L1-dcache-stores:u-square,L1-icache-loads:u,L1-icache-loads:u-square,L1-icache-load-misses:u,L1-icache-load-misses:u-square,dTLB-loads:u,...,mem-loads:u-square,mem-stores:u,mem-stores:u-square,ref-cycles:u,topdown-bad-spec:u,topdown-bad-spec:u-square,topdown-fe-bound:u,topdown-fe-bound:u-square,topdown-retiring:u,topdown-retiring:u-square
0,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,8.45,1.508973e+09,2.276998e+18,6.066891e+08,3.680716e+17,1.436276e+09,2.062890e+18,2.813354e+06,7.914960e+12,1.507874e+09,...,0.000000e+00,5.996344e+08,3.595614e+17,3.096516e+09,8.361428e+08,6.991347e+17,2.035314e+09,4.142504e+18,7.357896e+09,5.413863e+19
2,7.26,6.174994e+08,3.813055e+17,2.788412e+08,7.775240e+16,8.123170e+08,6.598588e+17,5.543507e+05,3.073047e+11,6.221361e+08,...,3.244042e+13,2.782868e+08,7.744352e+16,3.453882e+09,1.079781e+09,1.165927e+18,1.106342e+09,1.223993e+18,4.973468e+09,2.473539e+19
3,7.68,1.036649e+09,1.074641e+18,1.036638e+09,1.074618e+18,2.244483e+09,5.037702e+18,1.193041e+04,1.423346e+08,1.037553e+09,...,0.000000e+00,1.036512e+09,1.074357e+18,3.462589e+09,4.591660e+08,2.108334e+17,3.805071e+08,1.447857e+17,1.050205e+10,1.102930e+20
4,7.00,1.830498e+08,3.350721e+16,1.830314e+08,3.350048e+16,1.832026e+08,3.356321e+16,9.597917e+03,9.212001e+07,1.827194e+08,...,2.292091e+15,1.824524e+08,3.328887e+16,3.453620e+09,1.524338e+06,2.323606e+12,9.356647e+07,8.754684e+15,4.028857e+09,1.623169e+19


## Korrelationsmatrix

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

,watt-median,L1-dcache-loads:u,L1-dcache-loads:u-square,L1-dcache-stores:u,L1-dcache-stores:u-square,L1-icache-loads:u,L1-icache-loads:u-square,L1-icache-load-misses:u,L1-icache-load-misses:u-square,dTLB-loads:u,dTLB-loads:u-square,dTLB-load-misses:u,dTLB-load-misses:u-square,dTLB-stores:u,dTLB-stores:u-square,branch-loads:u,branch-loads:u-square,branch-load-misses:u,branch-load-misses:u-square,branch-instructions:u,branch-instructions:u-square,branch-misses:u,branch-misses:u-square,cache-misses:u,cache-misses:u-square,cache-references:u,cache-references:u-square,cpu-cycles:u-square,instructions:u,instructions:u-square,mem-loads:u,mem-loads:u-square,mem-stores:u,mem-stores:u-square,ref-cycles:u,topdown-bad-spec:u,topdown-bad-spec:u-square,topdown-fe-bound:u,topdown-fe-bound:u-square,topdown-retiring:u,topdown-retiring:u-square
watt-median,1.000000,0.423071,0.246619,0.367177,0.260107,0.431859,0.281022,0.164221,0.160904,0.418663,0.236945,0.160867,0.160477,0.366216,0.259299,0.364272,0.228870,0.144219,0.074764,0.366502,0.230853,0.142499,0.073321,0.162439,0.160481,0.160661,0.160477,0.848347,0.466999,0.344066,0.003264,0.002449,0.369547,0.264543,0.876825,0.142078,0.078215,0.120349,0.032247,0.485436,0.352194
L1-dcache-loads:u,0.423071,1.000000,0.861010,0.798494,0.660537,0.347326,0.198760,0.154669,0.154961,0.999560,0.848455,0.154899,0.154840,0.803739,0.674592,0.383851,0.220423,0.090100,0.045443,0.404505,0.235194,0.088872,0.044619,0.155141,0.154841,0.154869,0.154840,0.068024,0.434897,0.366181,-0.033435,-0.028095,0.789683,0.643174,0.084450,0.075358,0.026189,-0.000829,-0.100992,0.361622,0.286681
L1-dcache-loads:u-square,0.246619,0.861010,1.000000,0.664311,0.667411,0.194656,0.103142,0.073481,0.077978,0.871168,0.999433,0.078942,0.078930,0.676539,0.693928,0.203090,0.102401,-0.010114,-0.012608,0.234264,0.124187,-0.010528,-0.012739,0.078922,0.078929,0.078892,0.078930,0.027643,0.337582,0.316191,-0.060180,-0.043875,0.644446,0.629431,0.037166,-0.033925,-0.027676,-0.060663,-0.078509,0.246944,0.198658
L1-dcache-stores:u,0.367177,0.798494,0.664311,1.000000,0.924191,0.409208,0.314691,0.066180,0.067890,0.798157,0.653558,0.068412,0.068406,0.999573,0.923238,0.476692,0.353879,0.056846,0.094427,0.491001,0.364022,0.057452,0.095392,0.068581,0.068407,0.068389,0.068406,0.065846,0.480835,0.449041,-0.066736,-0.032364,0.999256,0.926590,0.076297,0.000333,0.060727,0.126758,0.004335,0.436201,0.400882
L1-dcache-stores:u-square,0.260107,0.660537,0.667411,0.924191,1.000000,0.334531,0.282185,-0.000656,0.003867,0.664309,0.661790,0.005098,0.005120,0.925024,0.998850,0.374419,0.284229,-0.023922,0.023142,0.391684,0.296125,-0.023358,0.023838,0.005053,0.005119,0.005054,0.005120,0.039277,0.420513,0.419142,-0.106611,-0.071270,0.919396,0.997876,0.043244,-0.081561,-0.005191,0.130386,0.051405,0.373806,0.358791
L1-icache-loads:u,0.431859,0.347326,0.194656,0.409208,0.334531,1.000000,0.953655,0.136564,0.138455,0.343265,0.186450,0.138629,0.138562,0.406628,0.329849,0.828477,0.694084,0.247339,0.120933,0.828575,0.694363,0.245936,0.119354,0.139031,0.138563,0.138582,0.138562,0.151533,0.782465,0.759446,-0.259093,-0.212456,0.412558,0.342798,0.169804,0.245621,0.172178,0.243969,0.145632,0.784171,0.763901
L1-icache-loads:u-square,0.281022,0.198760,0.103142,0.314691,0.282185,0.953655,1.000000,0.102354,0.108339,0.196051,0.098153,0.109432,0.109429,0.311830,0.276343,0.808336,0.728006,0.179467,0.076075,0.807175,0.727181,0.178529,0.074940,0.109584,0.109429,0.109383,0.109429,0.064739,0.694794,0.702115,-0.205259,-0.155270,0.317730,0.290583,0.080452,0.176267,0.128453,0.205351,0.127276,0.693394,0.706386
L1-icache-load-misses:u,0.164221,0.154669,0.073481,0.066180,-0.000656,0.136564,0.102354,1.000000,0.988300,0.150516,0.066465,0.981741,0.981645,0.064535,-0.001856,0.193174,0.152151,0.067702,-0.007898,0.194335,0.154424,0.060902,-0.009606,0.982397,0.981651,0.982016,0.981646,-0.181774,0.077860,0.046506,-0.045954,-0.036619,0.065236,-0.001120,-0.083290,0.052038,-0.011601,0.073880,0.007101,0.074757,0.046497
L1-ic

## 1. Versuch (Regression)

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, max_error

# --------------------------------------------
# Kein Intercept-Modell: fit_intercept=False setzt den Intercept auf 0.
# --------------------------------------------

# Hyperparameter für Balance: Einflussfaktor und Exponent für Gewichtung der Fehler
alpha = 20.0       # Betonung der Ausreißer
exponent = 1.5     # leicht überlineare Gewichtung

# Zielvariable: erste Spalte enthält die zu schätzende Zahl
y = df.iloc[:, 0]
# Potenzielle Features: alle anderen Spalten
potential_features = df.drop(columns=[df.columns[0]])

# Auswahl der 6 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=6)
X_selected = selector.fit_transform(potential_features, y)
selected_feature_names = potential_features.columns[selector.get_support()]
X = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training des ungewichteten Modells ohne Intercept
base_model = LinearRegression(fit_intercept=False)
base_model.fit(X_train, y_train)

# Berechnung der Trainingsfehler für Gewichtung
def train_and_compute_errors(model, X, y):
    preds = model.predict(X)
    return np.abs(y - preds)

train_errors = train_and_compute_errors(base_model, X_train, y_train)
# Normieren und Nichtlinear transformieren
errors_norm = train_errors / train_errors.max()
sample_weights = 1 + alpha * np.power(errors_norm, exponent)

# Training des gewichteten Modells ohne Intercept
model_balanced = LinearRegression(fit_intercept=False)
model_balanced.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluation: ungewichtet vs. gewichtet
def evaluate(model, X, y):
    preds = model.predict(X)
    return {
        'MSE': mean_squared_error(y, preds),
        'Max Error': max_error(y, preds)
    }

results = []
for name, model in [('Ungewichtet', base_model), ('Gewichtet', model_balanced)]:
    metrics = evaluate(model, X_test, y_test)
    results.append({'Modell': name, **metrics})

result_df = pd.DataFrame(results)
print(result_df)

# Ausgabe der ausgewählten Features
print("Ausgewählte Features:", selected_feature_names.tolist())

# Ausgabe der Koeffizienten (kein Intercept)
print("--- Ungewichtetes Modell (kein Intercept) ---")
print("Koeffizienten:")
for feat, coef in zip(selected_feature_names, base_model.coef_):
    print(f"  {feat}: {coef}")

print("--- Gewichtetes Modell (kein Intercept) ---")
print("Koeffizienten:")
for feat, coef in zip(selected_feature_names, model_balanced.coef_):
    print(f"  {feat}: {coef}")

# --------------------------------------------
# Speichern der Modelle und Metadaten für ein externes Projekt
# --------------------------------------------
import joblib
# Speichere die Modelle
joblib.dump(base_model, 'unweighted_model.pkl')
joblib.dump(model_balanced, 'weighted_model.pkl')
# Speichere die Namen der ausgewählten Features und Hyperparameter
metadata = {
    'selected_features': selected_feature_names.tolist(),
    'alpha': alpha,
    'exponent': exponent,
    'fit_intercept': False
}
joblib.dump(metadata, 'model_metadata.pkl')

print("Modelle und Metadaten wurden gespeichert:")
print(" - unweighted_model.pkl")
print(" - weighted_model.pkl")
print(" - model_metadata.pkl")


        Modell       MSE  Max Error
0  Ungewichtet  0.029137   0.433996
1    Gewichtet  0.061235   0.597915
Ausgewählte Features: ['L1-dcache-loads:u', 'L1-icache-loads:u', 'cpu-cycles:u-square', 'instructions:u', 'ref-cycles:u', 'topdown-retiring:u']
--- Ungewichtetes Modell (kein Intercept) ---
Koeffizienten:
  L1-dcache-loads:u: 3.8236487970396393e-10
  L1-icache-loads:u: 1.0969595829891947e-10
  cpu-cycles:u-square: -1.894352235366682e-18
  instructions:u: -1.193010823540599e-10
  ref-cycles:u: 5.519559399766688e-09
  topdown-retiring:u: 1.745630689479643e-10
--- Gewichtetes Modell (kein Intercept) ---
Koeffizienten:
  L1-dcache-loads:u: 3.048792273507325e-10
  L1-icache-loads:u: 4.7155159305481095e-11
  cpu-cycles:u-square: -1.6217383792175577e-18
  instructions:u: -1.8876339894242007e-10
  ref-cycles:u: 5.001291010517235e-09
  topdown-retiring:u: 2.666471030028046e-10
Modelle und Metadaten wurden gespeichert:
 - unweighted_model.pkl
 - weighted_model.pkl
 - model_metadata.pkl
